In [804]:
from math import ceil
import time, re, ast, sys, urllib, time, random, string, requests
import pandas as pd
from bs4 import BeautifulSoup
from random import randint
from selenium.webdriver.firefox.webdriver import FirefoxProfile
from urllib.request import urlopen, Request
from urllib.error import HTTPError, URLError
from itertools import cycle 
from header_list import user_agent_list
from proxies_list import *
from INITIAL_SCRAPPER_FUNCTIONS import *
from LIST_DF_FUNCTIONS import *
from PARSER_FUNCTIONS import *

In [805]:
ua = user_agent_list
proxies = proxies_list_
prx_pool = proxie_random_pool
proxies = proxie_check(proxies)

1
{'origin': '12.183.155.91'}
0.8446402549743652
[]
2
{'origin': '12.2.202.242'}
1.342216968536377
[]
3
Skipping. Connnection error
0.08500981330871582
[3]
4
{'origin': '188.52.254.27'}
6.474175214767456
[3]
5
{'origin': '50.224.173.189'}
1.1448442935943604
[3]
6
{'origin': '50.224.173.189'}
0.9913425445556641
[3]
7
{'origin': '50.224.173.189'}
0.8754110336303711
[3]
8
{'origin': '64.19.116.82'}
4.003408432006836
[3]
9
Skipping. Connnection error
0.10511994361877441
[3, 9]
10
Skipping. Connnection error
0.24464178085327148
[3, 9, 10]
11
{'origin': '72.35.40.34'}
0.9725842475891113
[3, 9, 10]
12
Skipping. Connnection error
0.10581684112548828
[3, 9, 10, 12]
13
Skipping. Connnection error
0.09174799919128418
[3, 9, 10, 12, 13]
14
{'origin': '98.26.32.15'}
1.1541237831115723
[3, 9, 10, 12, 13]
15
{'origin': '140.227.60.114'}
2.1740779876708984
[3, 9, 10, 12, 13]
16
Skipping. Connnection error
1.289520025253296
[3, 9, 10, 12, 13, 16]
17
Skipping. Connnection error
0.16283440589904785
[3, 9

IndexError: list index out of range

In [806]:
len(proxies)

37

In [204]:
# zip_codes = pd.read_csv('../Data/main_zips.csv')
# zip_list = zip_codes['zip'].tolist()

In [264]:
# active_zip_url_list, sold_zip_url_list = gen_url_list(zip_list)

In [213]:
# a_main_df = pd.DataFrame(columns=['full_address', 'home_link'])
# s_main_df = pd.DataFrame(columns=['full_address', 'home_link'])

In [261]:
# a = randint(0, 3)
# time.sleep(random.uniform(0, 1)*((88/13)**.33)*a)
# active_zip_url_list, a_main_df, proxies = links_for_props(proxies, active_zip_url_list, a_main_df, ua)

In [309]:
# a = randint(0, 3)
# sold_zip_url_list, s_main_df, proxies = links_for_props(proxies, sold_zip_url_list, s_main_df, ua)
# time.sleep(random.uniform(0, 1)*((88/13)**.33)*a)

In [295]:
# a_main_df.to_csv('../Data/active_props_url_list.csv')
# s_main_df.to_csv('../Data/sold_props_url_list.csv')

In [652]:
# a_main_df = pd.read_csv('../Data/active_props_url_list.csv')
# s_main_df = pd.read_csv('../Data/sold_props_url_list.csv')

In [304]:
# a_main_df = a_main_df.drop_duplicates(subset='full_address')
# s_main_df = s_main_df.drop_duplicates(subset='full_address')

In [305]:
active_url_list = a_main_df.home_link.tolist()
sold_url_list = s_main_df.home_link.tolist()

In [813]:
def info_from_sold_property(url, hdr, proxy):

    soup = session_creator(proxy, ua, url)
    
    df = top_info_parser(soup)

#     home_dict = address_parser(soup)

#     top_info_dict = top_info_parser(soup)

#     public_info_dict = public_info_parser(soup)

#     school_dict = school_parser(soup)

#     all_home_feats = feats_parser(soup)

#     df = pd.DataFrame()

#     df = pd.concat([home_dict, top_info_dict, school_dict,
#                     public_info_dict, all_home_feats], axis=1)

    return df

In [814]:
feat_list = []
feat_values = []


In [817]:
header = random.sample(ua, 1)[0]
proxy = random.sample(proxies, 1)[0]
sold_url_list = s_main_df['home_link'].tolist()
# url = random.sample(active_url_list, 1)[0]
url = random.sample(sold_url_list, 1)[0]
print(proxy)
print(url)

sold_home_df = info_from_sold_property(url, header, proxy)

98.101.147.86:50259
https://www.redfin.com/FL/Port-Richey/11346-Versailles-Ln-34668/unit-B/home/48319510
[<div class="HomeInfo inline-block"><div class="top-stats"><h1 class="address inline-block"><span data-rf-test-id="abp-homeinfo-homeaddress" itemscope="" itemtype="http://schema.org/SingleFamilyResidence"><span class="adr" itemprop="address" itemscope="" itemtype="http://schema.org/PostalAddress"><span class="street-address" data-rf-test-id="abp-streetLine" itemprop="streetAddress" title="11346 Versailles Ln Unit B">11346 Versailles Ln Unit B </span><span class="citystatezip" data-rf-test-id="abp-cityStateZip"><span class="locality" itemprop="addressLocality">Port Richey<!-- -->, </span><span class="region" itemprop="addressRegion">FL</span> <span class="postal-code" itemprop="postalCode">34668</span></span></span><span itemprop="geo" itemscope="" itemtype="http://schema.org/GeoCoordinates"><meta content="28.322919" itemprop="latitude"/><meta content="-82.674633" itemprop="longitude

In [811]:
sold_home_df

,address,city,zip_code,state,description,mls_num,num_beds,num_baths,home_sqft,yr_blt,...,Seller Disclosure:,View:,Unit Location:,Legal Description:,Sq. Ft. Source:,Sq. Ft. (Main Level):,Possession:,ListingType:,Limited Representation:,Supplement Number:
1,156 Front St NE #460,Salem,97301,None,Luxury 4th floor condo. Westerly views o/Willa...,18258478,2.0,2.0,753.0,2011,...,Disclosure,"River, Territorial",Upper Floor,"RIVERS CONDOMINIUMS, LOT R-10",county,"1,753",Close Of Escrow,Exclusive Right To Sell,Full Service,1


In [812]:
sold_home_df.to_dict()

{'address': {1: '156 Front St NE #460'},
 'city': {1: ' Salem'},
 'zip_code': {1: '97301'},
 'state': {1: None},
 'description': {1: 'Luxury 4th floor condo. Westerly views o/Willamette River & Waterfront park w/floor-ceiling windows. Open concept featuring stylish kitchen w/soft close, quartz, lrg island, high-end appliances inc micro drawer all w/5 yr warranty. BRs w/extensive tile & wall hung comfort height toilets. Master boasts large walk in closet,  tiled shower & dual vanities. W/D, included. Front balcony w/gas & overhead heater. Water on both balconies. '},
 'mls_num': {1: '18258478'},
 'num_beds': {1: 2.0},
 'num_baths': {1: 2.0},
 'home_sqft': {1: 753.0},
 'yr_blt': {1: 2011},
 'status': {1: 'Sold'},
 'redfin_est': {1: 573814.0},
 'sold_price': {1: 716500.0},
 'elem_school_name': {1: 'Bush Elementary School'},
 'elem_school_grades': {1: ' K to 5'},
 'elem_school_rating': {1: '1'},
 'Beds': {1: '2'},
 'Baths': {1: '2'},
 'Sq. Ft.': {1: '1,753'},
 'Stories': {1: '—'},
 'Lot Si